In [25]:
import pandas as pd
import numpy as np

In [26]:
from pathlib import Path

In [27]:
dPath = Path("../docs/dumps")

In [28]:
import pickle

In [29]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [30]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [31]:
X_train = train_data.drop("Detected", axis=1)

In [32]:
y_train = train_data.Detected

In [33]:
X_valid = valid_data.drop("Detected", axis=1)

In [34]:
y_valid = valid_data.Detected

In [35]:
with open(dPath / "rf_exp_04_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [36]:
X_train = X_train[names] 

In [37]:
X_valid = X_valid[names]

In [38]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [39]:
from imblearn.over_sampling import ADASYN

In [40]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [41]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 54.5 s


In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [43]:
adb = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=50, random_state=42)

In [54]:
%time adb.fit(X_res, y_res)

Wall time: 7min 20s


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [55]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [56]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [57]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [58]:
print_score(adb)

[0.7311838269900847, 0.6688692663268935, 0.6063974395812252]


In [59]:
conf_matr(adb)

              precision    recall  f1-score   support

       False       0.37      0.48      0.42     16657
        True       0.81      0.73      0.77     50839

    accuracy                           0.67     67496
   macro avg       0.59      0.61      0.59     67496
weighted avg       0.70      0.67      0.68     67496



In [60]:
with open(dPath / "adb_exp_01.pkl", 'wb') as filename:
    pickle.dump(adb,filename)

In [61]:
names = X_train.columns

In [62]:
feat_importances = pd.Series(adb.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [63]:
feat_importances

MutatorClass                     0.073829
mminstanceVariablesReferenced    0.057197
numAssertInTM                    0.052347
mmvariablesReferenced            0.048652
mmnumberOfExpressions            0.047272
pploc                            0.041975
ppnumberOfClasses                0.041739
ppavcc                           0.038246
mmexternalMethodsCalled          0.035979
numTestCover                     0.035207
numExecuted                      0.033950
ccimportedPackages               0.032654
numAssertInTC                    0.031179
ccfanIn                          0.030213
pptcc                            0.027669
ppmaxcc                          0.027300
ppfanin                          0.026992
ppmaintainabilityIndex           0.026113
ppnumberOfMethods                0.025350
ccmaintainabilityIndex           0.024119
ppRVF                            0.023071
ppinstability                    0.022948
cchalsteadEffort                 0.021569
ppmaintainabilityIndexNC         0

In [64]:
with open(dPath / "adb_exp_01_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)